In [1]:
import altair as alt
import os
import pandas as pd
import glob
import tensorflow as tf

2022-10-17 11:26:49.874199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 11:26:50.177487: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-17 11:26:51.764281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-17 11:26:51.764405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [2]:
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [3]:
summary_iterator = tf.compat.v1.train.summary_iterator

def get_section_results(file, xtag='Train_EnvstepsSoFar', ytag='Eval_AverageReturn'):
    """
        requires tensorflow==1.12.0
    """

    X = []
    Y = []
    for e in summary_iterator(file):
        for v in e.summary.value:
            if v.tag == xtag:
                X.append(v.simple_value)
            elif v.tag == ytag:
                Y.append(v.simple_value)
    return X, Y
logdir = '../../data/q2_pg_q2*/events*'
glob.glob(logdir)


def make_dataframe(eventfiles, xtag='Train_EnvstepsSoFar', ytag='Eval_AverageReturn'):
    dfs = []
    for eventfile in eventfiles:
        dirname = os.path.basename(os.path.dirname(eventfile))
        X, Y = get_section_results(eventfile, xtag, ytag)
        n = len(X)
        msg = f"{(len(X), len(Y))}"
        length = min(len(X), len(Y))
        X = X[:length]
        Y = Y[:length]
        assert len(X) == len(Y), msg
        log_df = pd.DataFrame.from_dict({xtag: X, ytag: Y, 'exp': [dirname] * length})
        dfs.append(log_df)
    
    return pd.concat(dfs)

# Q1

In [4]:
logdir = '../../data/q1_*/events*'


In [5]:
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q1  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

chart = alt.Chart(df_q1).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


alt.Chart(...)

# Q2

In [43]:
logdir = '../../data/q2_*/events*'

In [44]:
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q2  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [45]:
double_qdn_rows = df_q2['exp'].str.contains('double') 

In [46]:
import numpy as np

In [47]:
df_q2['Model'] = np.where(double_qdn_rows, 'Double DQN', 'DQN')

In [48]:
df_q2.head()

,Train_EnvstepsSoFar,Train_AverageReturn,exp,Model
0,1.0,-269.984650,q2_dqn_1_LunarLander-v3_05-10-2022_17-35-55,DQN
1,10001.0,-215.556900,q2_dqn_1_LunarLander-v3_05-10-2022_17-35-55,DQN
2,20001.0,-198.865585,q2_dqn_1_LunarLander-v3_05-10-2022_17-35-55,DQN
3,30001.0,-183.948914,q2_dqn_1_LunarLander-v3_05-10-2022_17-35-55,DQN
4,40001.0,-172.239243,q2_dqn_1_LunarLander-v3_05-10-2022_17-35-55,DQN


In [49]:
# df_q2 = df_q2.groupby(['Model', 'Train_EnvstepsSoFar'], as_index=False)['Train_AverageReturn'].mean()

In [55]:
line = alt.Chart(df_q2).mark_line().encode(
    x=xtag,
    y=f'mean({ytag})',
    color='Model'
)
band = alt.Chart(df_q2).mark_errorband(extent='ci').encode(
    x=xtag,
    y=ytag,
    color='Model'
)
for chart in (line, band):
    chart = chart.configure_legend(
    labelLimit=1000
)
line + band

alt.LayerChart(...)

In [8]:
# TODO: Average results for DQN and Double DQN over seeds

# Q3

In [9]:
logdir = '../../data/q3_*/events*'

In [10]:
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q1  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

chart = alt.Chart(df_q1).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)

# Q4

In [11]:
logdir = '../../data/q4_*/events*'
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q4  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [12]:

chart = alt.Chart(df_q4).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)

# Q5

In [13]:
logdir = '../../data/q5_*Cheetah*/events*'
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q5 = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [14]:
chart = alt.Chart(df_q5).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)

In [15]:
logdir = '../../data/q5_*Invert*/events*'
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q5 = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [16]:
chart = alt.Chart(df_q5).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)

# Q6

In [17]:
logdir = '../../data/q6a_*/events*'
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q6 = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [18]:
chart = alt.Chart(df_q6).mark_line().encode(
    x=xtag,
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)

In [19]:
logdir = '../../data/q6b_*/events*'
eventfiles = glob.glob(logdir)
xtag = 'Train_EnvstepsSoFar'
ytag = 'Train_AverageReturn'
df_q6b = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

In [20]:
chart = alt.Chart(df_q6b).mark_line(clip=True).encode(
    x=alt.X(xtag, scale=alt.Scale(domain=(0, 200_000))),
    y=ytag,
    color='exp'
)
chart = chart.configure_legend(
    labelLimit=1000
)
chart

alt.Chart(...)